In [ ]:
"""
rankboost_churn_pipeline.py

Açıklama:
Bu dosya, müşteri işlem geçmişi verisini kullanarak basit bir churn tahmin hattı (pipeline) oluşturmaktadır.

Girdi dosyaları (örnek):
- customer_history.csv      : Günlük/aylık müşteri işlem geçmişi ("date" ve "cust_id" içerir)
- referance_data_test.csv   : Test seti ("ref_date" içerir)
- sample_submission.csv     : Çıktı şablonu ("cust_id" ve muhtemelen beklenen hedef sütunu içerir)

Çıktı:
- submission.csv  : Hazırlanan tahminleri içeren csv

Notlar / Gereksinimler:
- scikit-learn, pandas, numpy, xgboost, lightgbm kütüphaneleri yüklü olmalıdır.
- Bu pipeline eğitim verisinden proxy churn label (aktiflik tabanlı) oluşturur; gerçek etiketler yoksa bu yaklaşım kullanılabilir.
- Hyperparametreler örnektir; gerçek kullanımda cross-validation veya zaman serisi validasyonu ile ayarlanmalıdır.

"""

# 1) Kütüphaneler
# Veri işleme için pandas & numpy, ölçekleme ve veri bölme için sklearn,
# modeller için XGBoost ve LightGBM.
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression

# 2) Dosya yolları
train_path = "customer_history.csv"
test_path = "referance_data_test.csv"
sample_path = "sample_submission.csv"
out_path = "submission_rankboost.csv"

# 3) Veri yükleme
# Tarih sütunlarını parse_dates ile yükleyerek zaman aritmetiğini kolaylaştırılır.
# "date" ve "ref_date" isimleri datasetinizde farklıysa burada uyarlanmalıdır.
train = pd.read_csv(train_path, parse_dates=["date"])  # eğitim (işlem geçmişi)
test = pd.read_csv(test_path, parse_dates=["ref_date"])  # referans/test tarihleri
sample = pd.read_csv(sample_path)  # submission şablonu (cust_id listesi vb.)


# 4) Otomatik sütun seçimi: transaction count / amount sütunları
# Train setinde isimlendirme standartı varsa (_cnt, _amt gibi) bu kısım otomatik
# sütunları algılar. Eğer farklıysa burada manuel tanımlayın.
txn_cnt_cols = [c for c in train.columns if c.endswith("_cnt")]
txn_amt_cols = [c for c in train.columns if c.endswith("_amt")]

# ref_date: test dosyasındaki en son referans tarihini alınır. Bu tarih "snapshot" tarihi
# olarak kullanılacak ve tüm pencere hesaplamaları bu tarihe göre yapılacaktır.
ref_date = test["ref_date"].max()

# 5) Özellik oluşturucu fonksiyon (feature engineering)
# make_features: verilen bir aylık pencere (months) için her müşteri bazında
# toplu istatistikler üretir (sum, mean, std, max, min). Ayrıca aktif ürün sayısı
# gibi kategorik/unique bilgi de çıkarılır.
# Bu fonksiyonun davranışı dataset yapısına göre uyarlanabilir (ek istatistikler,
# daha sofistike zaman serisi özellikleri vs.).
def make_features(df, ref_date, months):
    # Başlangıç zamanı: ref_date - months
    start = ref_date - pd.DateOffset(months=months)

    # Pencere içindeki kayıtları seç
    sub = df[(df["date"] > start) & (df["date"] <= ref_date)].copy()

    # Aggregasyon sözlüğü: her sayısal işlem sütunu için birden fazla istatistik
    agg = sub.groupby("cust_id").agg({
        **{c: ["sum", "mean", "std", "max", "min"] for c in txn_cnt_cols + txn_amt_cols},
        # aktif ürün kategorisi adedi: eşsiz sayısı ve maksimumu (örnek amaçlı)
        "active_product_category_nbr": ["nunique", "max"]
    })

    # Çok seviyeli sütun isimlerini okunabilir tek seviyeye indirgeme
    agg.columns = [f"{a}_{b}_{months}m" for a,b in agg.columns]
    return agg.reset_index()

# 6) Çoklu pencere (multi-window) özetleri
# Farklı zaman pencereleri (3, 6, 12 ay) için özellikler hesaplanır. Bu, kısa ve uzun
# dönem davranışlarını yakalamaya yardımcı olur.
agg3 = make_features(train, ref_date, 3)
agg6 = make_features(train, ref_date, 6)
agg12 = make_features(train, ref_date, 12)

# Merge: Tüm özellik setlerini cust_id üzerinden birleştiriyoruz. Eksik değerler 0 ile doldurulur.
features = agg12.merge(agg6, on="cust_id", how="outer").merge(agg3, on="cust_id", how="outer").fillna(0)

# 7) Trend & delta (oransal özellikler)
# Örnek: 6 aylık toplam / 12 aylık toplam => son yarı dönemde hareket var mı?
# 3 aylık / 6 aylık => kısa dönem değişim.
for c in txn_cnt_cols + txn_amt_cols:
    # 6m / 12m trend
    if f"{c}_sum_6m" in features.columns and f"{c}_sum_12m" in features.columns:
        # +1 ile bölme, sıfıra bölünmeyi ve logik oranları stabilize eder
        features[f"{c}_trend"] = (features[f"{c}_sum_6m"] + 1) / (features[f"{c}_sum_12m"] + 1)
    # 3m / 6m delta
    if f"{c}_sum_3m" in features.columns and f"{c}_sum_6m" in features.columns:
        features[f"{c}_delta"] = (features[f"{c}_sum_3m"] + 1) / (features[f"{c}_sum_6m"] + 1)

# 8) Aktivite skoru ve proxy churn label oluşturma
# Gerçek churn etiketleri yoksa, proxy olarak "aktiflik" bazlı bir kural tanımlanabilir.
# Burada tüm "sum_" ile biten özellikleri topluyor, yüzde sıralama (rank pct) yapıyor ve
# en düşük %25'i churn (1) olarak işaretliyoruz. Bu, basit ama etkili bir zayıf süpervizyon yöntemidir.
activity_cols = [c for c in features.columns if "sum_" in c]
features["activity_sum"] = features[activity_cols].sum(axis=1)
features["activity_rank"] = features["activity_sum"].rank(pct=True)

# churn_label: düşük aktivite -> churn. 0 = aktif, 1 = churn (hedef değişimi gerektirebilir)
features["churn_label"] = (features["activity_rank"] < 0.25).astype(int)  # en düşük %25 churn

# 9) Ölçekleme (scaling) ve X/y ayrımı
# RobustScaler: aykırı değerlere (outliers) karşı daha dayanıklı olduğu için tercih edildi.
X = features.drop(columns=["cust_id","churn_label"])  # model girdileri
y = features["churn_label"]  # hedef
scaler = RobustScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# 10) Train/validation split
# Basit bir rastgele bölme. Zaman serisi problemlerinde zaman tabanlı validasyon veya
# cross-validation kullanmak genelde daha doğrudur. Burada örnek amaçlı rastgele split kullanıldı.
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 11) XGBoost ile eğitim
# Parametreler örnektir. Daha iyi sonuç için hyperparametre optimizasyonu önerilir.
xgb_model = XGBClassifier(
    n_estimators=800,
    max_depth=6,
    learning_rate=0.02,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss",
    n_jobs=-1,
    random_state=42
)
# eval_set ile validation üzerinde izlenebilir; verbose ile dönemlik çıktı alınır.
xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=100)

# 12) LightGBM ile eğitim
# LightGBM için Dataset objeleri hazırlanır. callbacks kullanarak early stopping ve loglama eklenir.
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_val = lgb.Dataset(X_val, label=y_val)
params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "learning_rate": 0.02,
    "num_leaves": 96,
    "feature_fraction": 0.85,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "min_data_in_leaf": 30,
    "seed": 42,
    "verbosity": -1
}

# early_stopping: validation'da 100 round boyunca düzelme olmazsa durdur.
callbacks = [lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(period=200)]
# num_boost_round yüksek verildi; early stopping ile gerçek uygun iterasyon seçilecek.
lgb_model = lgb.train(params, lgb_train, valid_sets=[lgb_val], num_boost_round=1500, callbacks=callbacks)

# 13) Test için özellik seti oluşturma
# submission için testteki cust_id'lerle eşleşen features satırları alınır. Eğer test setinde
# ekstra müşteriler varsa veya zaman pencerelemesi farklıysa burada ek adımlar gerekir.
test_feats = features[features["cust_id"].isin(sample["cust_id"])].copy()
X_test = pd.DataFrame(scaler.transform(test_feats.drop(columns=["cust_id","churn_label"])), columns=X.columns)

# 14) Model tahminleri
# XGBoost predict_proba ile olasılık alınır; LightGBM için predict kullanılır.
p_xgb = xgb_model.predict_proba(X_test)[:,1]
p_lgb = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)

# 15) Ağırlıklı blend (ensemble)
# İki modelin ağırlıklı ortalaması alınır. Ağırlıklar ihtiyaca göre değiştirilebilir.
blend = 0.6*p_lgb + 0.4*p_xgb

# 16) Rank-based calibration (top boosting)
# Bu adım rank tabanlı küçük bir hack: üst %10 ve %5'lik dilimlere ekstra multiplier uygulayarak
# "rank boost" yapıyoruz. Bu tip ayarlamalar leaderboard odaklı yarışmalarda yaygındır fakat
# gerçek iş uygulamalarında dikkatli kullanılmalıdır (overfitting riski).
rank = pd.Series(blend).rank(pct=True)
boosted = blend.copy()
# Üst %10'u %50 arttır (1.5), üst %5'i daha da arttırarak (1.8)
boosted[rank > 0.9] *= 1.5
boosted[rank > 0.95] *= 1.8
boosted = np.clip(boosted, 0, 1)  # olasılıkları [0,1] aralığına zorla

# 17) Submission oluşturma ve kaydetme
sub = sample.copy()

# sample dosyasında hedef sütun adı farklıysa (ör. "target" veya "churn") burada uyarlayın.
sub["churn"] = boosted
sub.to_csv(out_path, index=False)

print(f"\u2705 Final rank-boosted submission kaydedildi: {out_path}")
print(sub.head())

[0]	validation_0-logloss:0.54459
[100]	validation_0-logloss:0.05417
[200]	validation_0-logloss:0.00863
[300]	validation_0-logloss:0.00262
[400]	validation_0-logloss:0.00179
[500]	validation_0-logloss:0.00169
[600]	validation_0-logloss:0.00167
[700]	validation_0-logloss:0.00167
[799]	validation_0-logloss:0.00169
Training until validation scores don't improve for 100 rounds
[200]	valid_0's binary_logloss: 0.00890564
[400]	valid_0's binary_logloss: 0.00235914
Early stopping, best iteration is:
[399]	valid_0's binary_logloss: 0.00235382
✅ Final rank-boosted submission kaydedildi: submission_rankboost.csv
   cust_id     churn
0        1  0.000062
1        2  0.000055
2        9  0.000057
3       15  0.000056
4       19  0.000053
